In [1]:
# 1. Imports
import numpy as np
import matplotlib.pyplot as plt
import pyshtools as pysh
from pyshtools import constants

# 2. Load the EGM2008 Gravitational Model coefficients
print("Attempting to load EGM2008 model through module dataset...")
try:
    # The 'Earth' submodule contains the gravity model.
    # The call returns the SHGravCoeffs object directly.
    clm = pysh.datasets.Earth.EGM2008()
    print("Model loaded and SHCoeffs object initialized successfully.")

except Exception as e:
    print(f"CRITICAL ERROR: {e}")
    print("The automatic download/initialization failed. Check network connection.")
    raise

# 3. Verify the loaded coefficients
print(f"Model coefficients structured up to degree/order {clm.lmax}.")

# 4. Confirm Earth's reference parameters
GM = clm.gm
R0 = clm.r0
print(f"Reference GM: {GM} m^3/s^2")
print(f"Reference Radius (R0): {R0:.3f} meters")

Attempting to load EGM2008 model through module dataset...
Model loaded and SHCoeffs object initialized successfully.
Model coefficients structured up to degree/order 2190.
Reference GM: 398600441500000.0 m^3/s^2
Reference Radius (R0): 6378136.300 meters


In [2]:
# --- PHASE 2.1 & 2.2: Local Execution Memory Optimized ---

print("Starting memory-optimized calculation...")

# 1. Isolate the Anomalous Coefficients (Filter l=0 and l=1)
clm_anomalous = clm.copy()
clm_anomalous.coeffs[:, 0:2, :] = 0.0
print(f"Coefficients set to zero for degrees 0 and 1 via 3D slicing. Lmax is: {clm_anomalous.lmax}")

# 2. Define the reference ellipsoid parameters using the correct constants
a = constants.Earth.wgs84.a.value
f = constants.Earth.wgs84.f.value
omega = constants.Earth.wgs84.omega.value
u0 = constants.Earth.wgs84.u0.value
print(f"Reference Geoid Potential (U0): {u0} m²/s²")

# We use lmax=720 for a good balance of detail and speed for grid calculation.
LMAX_PLOT = 90

# 3. Calculate the Geoid Height Grid (Delta N)
grid_geoid_object = clm_anomalous.geoid(
    potref=u0,
    a=a,
    f=f,
    omega=omega,
    r=clm.r0, # Use the reference radius from the loaded coefficient file
    lmax=LMAX_PLOT,
    grid='DH2')
print(f"Geoid height grid object calculated up to degree {LMAX_PLOT}.")

# # 4. Calculate the Free-Air Gravity Anomaly Grid (Delta g)
# grid_gravity_object = clm_anomalous.expand(
#     lmax=LMAX_PLOT,
#     a=a,
#     f=f,
#     grid='DH')
# print(f"Gravity Disturbance grid object calculated up to degree {LMAX_PLOT}.")

# # 5. Extract, Convert Data Type, and Create DataFrame
# grid_geoid_data = grid_geoid_object.geoid
# grid_gravity_data = grid_gravity_object.total

# # CRITICAL MEMORY STEP: Convert the large NumPy arrays to 32-bit floats
# # This reduces the memory footprint by 50% for the final data arrays.
# geoid_data_32 = grid_geoid_data.astype(np.float32)
# gravity_data_32 = grid_gravity_data.astype(np.float32)

# # Extract coordinates
# lats_2d = grid_geoid_object.geoid.lats() 
# lons_2d = grid_geoid_object.geoid.lons() 

# # Create DataFrame
# df = pd.DataFrame({
#     'Latitude': lats_2d.flatten(),
#     'Longitude': lons_2d.flatten(),
#     'Geoid_m': geoid_data_32.flatten(),       
#     'Gravity_mGal': gravity_data_32.flatten() * 10**5 # Convert to m/s^2 to mGal
# })

# print("All grids calculated and DataFrame successfully created locally.")
# print(f"DataFrame size: {len(df)} points. Geoid Min/Max: {df['Geoid_m'].min():.2f} / {df['Geoid_m'].max():.2f} meters.")
# print(df.head())

Starting memory-optimized calculation...
Coefficients set to zero for degrees 0 and 1 via 3D slicing. Lmax is: 2190
Reference Geoid Potential (U0): 62636851.7146 m²/s²
Geoid height grid object calculated up to degree 90.


In [ ]:
# # --- PHASE 2.1: Execution of Grid Calculation ---

# # 1. Isolate the Anomalous Coefficients (Filter l=0 and l=1)
# clm_anomalous = clm.copy()
# clm_anomalous.coeffs[:, 0:2, :] = 0.0

# print(f"Coefficients set to zero for degrees 0 and 1 via 3D slicing. Lmax is: {clm_anomalous.lmax}")

# # 2. Define the reference ellipsoid parameters
# a = constants.Earth.wgs84.a.value
# b = constants.Earth.wgs84.b.value
# f = constants.Earth.wgs84.f.value
# omega = constants.Earth.wgs84.omega.value
# u0 = constants.Earth.wgs84.u0.value

# # 3. Calculate the Geoid Height Grid (Delta N)
# lmax_plot = 360
# grid_geoid_object = clm_anomalous.geoid(
#     potref=u0,
#     a=a,
#     f=f,
#     omega=omega,
#     lmax=lmax_plot,
#     grid='DH')
# print(f"Geoid height grid object calculated up to degree {lmax_plot}.")

# # 4. Calculate the Free-Air Gravity Anomaly Grid (Delta g)
# grid_gravity_object = clm_anomalous.expand(
#     lmax=lmax_plot,
#     a=a,
#     f=f,
#     grid='DH')
# print(f"Gravity Disturbance grid object calculated up to degree {lmax_plot}.")

# # Final step: Extract the data arrays for plotting (Data Science Step)
# grid_geoid_data = grid_geoid_object.geoid
# grid_gravity_data = grid_gravity_object.total
# print("Raw data arrays extracted for plotting.")

Coefficients set to zero for degrees 0 and 1 via 3D slicing. Lmax is: 2190
